In [55]:
import pandas as pd
xls=pd.ExcelFile('Paavan Dairy-automated-invoice.xlsx')
client_details=pd.read_excel(xls,'Clients')
product_details=pd.read_excel(xls,'Products')
Bill=pd.read_excel(xls,'Sheet1')

In [56]:
final_bill={}
flag=0
for index,row in Bill.iterrows():
    client_id=row['Client Number']
    if client_id=='nan':
        break
    product_id=row['Product']
    quantity=row['Quantity']
    name=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'Name'])[1:]).strip()
    name=(name.split('\n'))[0]
    street=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'Street'])[1:]).strip()
    street=(street.split('\n'))[0]
    city=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'City'])[1:]).strip()
    city=(city.split('\n'))[0]
    Phone_Number=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'Phone Num'])[1:]).strip()
    Phone_Number=(Phone_Number.split('\n'))[0]
    Product=(str(product_details.loc[product_details['Product nr']==row['Product'],'Description'])[1:]).strip()
    Product=(Product.split('\n'))[0]
    Price=(str(product_details.loc[product_details['Product nr']==row['Product'],'Price'])[1:]).strip()
    Price=(Price.split('\n'))[0]
    if row['Client Number'] not in final_bill.keys():
        prod={}
        prod[0]=({'product_id':product_id,'Product':Product,'Price':Price,'Quantity':quantity})
        final_bill[row['Client Number']]={'Client_Number':row['Client Number'],'Client_Name':name,'Street':street,'City':city,'Phone_Number':Phone_Number,'prod':prod}
    else:
        prod={}
        prod=final_bill[row['Client Number']]['prod']
        product_number=prod.keys()
        count=len(product_number)
        for j in range(count):
            if prod[j]['product_id']==product_id:
                final_bill[row['Client Number']]['prod'][j]['Quantity']+=quantity
                flag=1
                break
        #print(name+str("")+str(flag))
        if flag==1:
            flag=0
        else:
            prod[count]=({'product_id':product_id,'Product':Product,'Price':Price,'Quantity':quantity})
            final_bill[row['Client Number']]={'Client_Number':row['Client Number'],'Client_Name':name,'Street':street,'City':city,'Phone_Number':Phone_Number,'prod':prod}
        

In [57]:
df=(pd.DataFrame(final_bill))

In [58]:
df=df.transpose()

In [59]:
for index,j in df.iterrows():
    j['prod']=(pd.DataFrame(j['prod']))

In [60]:
df=df.transpose()

In [61]:
df

,501,502,503
City,Asansol,Asansol,Asansol
Client_Name,Pratik Bansal,Simarjit Gandhi,G B Gupta
Client_Number,501,502,503
Phone_Number,9647616532,9570109807,9999999999
Street,"Bansal Bhawan, G.T. Road, Kumarpur","Lucas Estate, Durgamandir",Sreepally
prod,0 1 2 Price ...,0 1 Price 20 ...,0 Price 38 Product ...


In [24]:
df[501]['prod']

,0,1,2
Price,50,38,20
Product,Bottle,Milk,Dahi
Quantity,0,0,1
product_id,1002,1001,1003


In [25]:
for index,i in (df[501]['prod'].transpose()).iterrows():
    print(i['Price'])

50
38
20


In [39]:
import pdfkit
#import wkhtmltopdf

In [43]:
import json
import random
import datetime
import pandas as pd
from pprint import pprint
file=open(r'rajat_bill.html','r')
file1=open(r'bill_gen2.html','w')
content=file.read()

In [44]:
content=content.replace('\t',"")
content = content.replace("\n","")
    

In [45]:
for val,row in df.iterrows():
    client_id=row['Client_Number']
    cid=str(row['Client_Number'])
    city=str(row['City'])
    name=str(row['Client_Name'])
    Phone=str(row['Phone_Number'])
    street=str(row['Street'])
    
prototype=""
to ="to :"
findto=content.find(to)
prototype+=content[0:findto+len(to)]+" "+name+"<br>"+street+"<br>"+

date = "Date :"
finddate=content.find(date)
now = datetime.datetime.now()
prototype+=content[findto+len(to):finddate+len(date)]+" "+str(now.date())

month ="Month :"
findmonth=content.find(month)
prototype+=content[finddate+len(date):findmonth+len(month)]+" "+str(now.strftime("%B"))

invoicenr = "Invoice nr :"
findinvoicenr = content.find(invoicenr)
prototype+=content[findmonth+len(month):findinvoicenr+len(invoicenr)]+" "+str(random.randint(1,100))

clientnr ="Client nr :"
findclientnr=content.find(clientnr)
prototype+=content[findinvoicenr+len(invoicenr):findclientnr+len(clientnr)]+" "+"501"

tbody="tbody"
findtbody=content.find(tbody)
prototype+=content[findclientnr+len(clientnr):findtbody+len(tbody)]

total_amount = 0
for index,i in (df[501]['prod'].transpose()).iterrows():
    myprice=str(i['Price'])
    myproduct=str(i['Product'])
    mypid=str(i['product_id'])
    myquantity=str(i['Quantity'])
    total = int(myquantity)*float(myprice)
    total_amount = total_amount+int(total)
    prototype+="<tr><td>"+mypid+"</td>"+"<td>"+myproduct+"</td>"+"<td>"+myprice+"</td>"+"<td>"+myquantity+"</td>"+"<td>"+str(total)+"</td></tr>"

prototype+="</tbody></table></div>"
prototype+='<br><br><h3 class="float-right">Total Amount Due : &#x20B9; '+str(total_amount)+"</div></body></html>"

file1.write(prototype)
pdfkit.from_file('bill_gen2.html', 'rajatjain.pdf')
file1.close()


OSError: No wkhtmltopdf executable found: "b''"
If this file exists please check that this process can read it. Otherwise please install wkhtmltopdf - https://github.com/JazzCore/python-pdfkit/wiki/Installing-wkhtmltopdf

In [46]:
"""
HTMLTextToPDF.py
A demo program to show how to convert the text extracted from HTML 
content, to PDF. It uses the Beautiful Soup library, v4, to 
parse the HTML, and the xtopdf library to generate the PDF output.
Beautiful Soup is at: http://www.crummy.com/software/BeautifulSoup/
xtopdf is at: https://bitbucket.org/vasudevram/xtopdf
Guide to using and installing xtopdf: http://jugad2.blogspot.in/2012/07/guide-to-installing-and-using-xtopdf.html
Author: Vasudev Ram - http://www.dancingbison.com
Copyright 2015 Vasudev Ram
"""

import sys
from bs4 import BeautifulSoup
from PDFWriter import PDFWriter

def usage():
    sys.stderr.write("Usage: python " + sys.argv[0] + " html_file pdf_file\n")
    sys.stderr.write("which will extract only the text from html_file and\n")
    sys.stderr.write("write it to pdf_file\n")

def main():

    pw=DFWriter("RajatJain.pdf")
 
    # Use method chaining this time.
    for line in BeautifulSoup(prototype).get_text().split("\n"):
        pw.writeLine(line)
    pw.savePage()
    pw.close()
if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'PDFWriter'